# Simulation of Secure Multiparty Computation Framework

### How does it work?
It works like this: your device downloads the current model, improves it by learning from data on your phone, and then summarizes the changes as a small focused update. Only this update to the model is sent to the cloud, using encrypted communication, where it is immediately averaged with other user updates to improve the shared model. All the training data remains on your device, and no individual updates are stored in the cloud.
- https://ai.googleblog.com/2017/04/federated-learning-collaborative.html
- https://www.youtube.com/watch?v=89BGjQYA0uE Desde el 13:50 Hasta 19:26

Hi,
You recently answered me in the OpenFL githhub with 3 examples from your repos. I want to connect with you as I see your profile very interesting and with great knowledge about FL. I would like to connect with you as I am developing my university thesis and I think I could learn a lot from you.

## Libraries

In [1]:
import torch
import os
import glob
import torchvision
#import cv2
import tqdm as tqdm

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import openfl.native as fx
import matplotlib.pyplot as plt
from torchvision import models,datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from openfl.interface.interactive_api.federation import Federation
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import precision_recall_fscore_support
from copy import deepcopy
from collections import OrderedDict


c:\Users\ddelriod\Anaconda3\envs\TFG-FL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client_id = 'api'
cert_dir = 'cert'
director_node_fqdn = 'localhost'
director_port = '50051'

federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port, 
    tls=False
)

In [3]:
federation.target_shape

['150', '150', '1']

In [4]:
shard_registry = federation.get_shard_registry()
shard_registry

{'env_one': {'shard_info': node_info {
    name: "env_one"
  }
  shard_description: "Chest X-ray dataset, shard number 1 out of 2"
  sample_shape: "150"
  sample_shape: "150"
  sample_shape: "1"
  target_shape: "150"
  target_shape: "150"
  target_shape: "1",
  'is_online': True,
  'is_experiment_running': False,
  'last_updated': '2023-03-07 01:33:17',
  'current_time': '2023-03-07 01:33:54',
  'valid_duration': seconds: 120,
  'experiment_name': 'ExperimentName Mock'},
 'env_two': {'shard_info': node_info {
    name: "env_two"
  }
  shard_description: "Chest X-ray dataset, shard number 2 out of 2"
  sample_shape: "150"
  sample_shape: "150"
  sample_shape: "1"
  target_shape: "150"
  target_shape: "150"
  target_shape: "1",
  'is_online': True,
  'is_experiment_running': False,
  'last_updated': '2023-03-07 01:32:57',
  'current_time': '2023-03-07 01:33:54',
  'valid_duration': seconds: 120,
  'experiment_name': 'ExperimentName Mock'}}

In [5]:
dummy_shard_desc = federation.get_dummy_shard_descriptor(size=2)
dummy_shard_dataset = dummy_shard_desc.get_dataset('train')
sample, target = dummy_shard_dataset[0]
print(sample.shape)
print(target.shape)

(150, 150, 1)
(150, 150, 1)


## Data Preparation

In [6]:
class TransformedDataset(Dataset):
    """Image Person ReID Dataset."""

    def __init__(self, dataset, transform=None, target_transform=None):
        """Initialize Dataset."""
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        """Length of dataset."""
        return len(self.dataset)

    def __getitem__(self, index):
        img, label = self.dataset[index]
        label = self.target_transform(label) if self.target_transform else label
        img = self.transform(img) if self.transform else img
        return img, label

In [7]:
class ChestXrayDataset(DataInterface):
    def __init__(self, **kwargs):
        self.kwargs = kwargs
    
    @property
    def shard_descriptor(self):
        return self._shard_descriptor
        
    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        
        self.train_set = TransformedDataset(
            self._shard_descriptor.get_dataset('train'),
            transform=None
        )
        self.valid_set = TransformedDataset(
            self._shard_descriptor.get_dataset('val'),
            transform=None
        )
        
    def get_train_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        generator=torch.Generator()
        generator.manual_seed(0)
        return DataLoader(
            self.train_set, batch_size=self.kwargs['train_bs'], shuffle=True, generator=generator
            )

    def get_valid_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        return DataLoader(self.valid_set, batch_size=self.kwargs['valid_bs'])

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        return len(self.train_set)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.valid_set)

In [8]:
fed_dataset = ChestXrayDataset(train_bs=32, valid_bs=32)

## Describe the model and optimizer

In [ ]:
model_net = models.densenet121(pretrained=True) # we will use a pretrained model and we are going to change only the last layer
for param in model_net.parameters():
    param.requires_grad = True
    
model_net.classifier = nn.Sequential(OrderedDict([
    ('fcl1', nn.Linear(1024,256)),
    ('dp1', nn.Dropout(0.3)),
    ('r1', nn.ReLU()),
    ('fcl2', nn.Linear(256,32)),
    ('dp2', nn.Dropout(0.3)),
    ('r2', nn.ReLU()),
    ('fcl3', nn.Linear(32,2)),
    ('out', nn.LogSoftmax(dim=1)),
]))

In [10]:
params_to_update = []
for param in model_net.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
'''
FEDPROX
'''        
#from openfl.utilities.optimizers.torch import FedProxAdam        
#optimizer = FedProxAdam(params_to_update, lr=1e-4, mu=0.01)

'''
ORIGINALE
'''
optimizer = optim.Adadelta(params_to_update, lr = 0.05)

#optimizer = optim.AdamW(params_to_update, lr=0.001, weight_decay=0.02)
#optimizer = optim.SGD(params_to_update, lr=0.01, momentum=0.9, weight_decay=0.0005)

#scheduler
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
def binary_cross_entropy(output, target):
    """Cross-entropy metric
    """
    #return F.cross_entropy(input=output,target=target)
    #return F.binary_cross_entropy_with_logits(input=output,target=target)
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(output, target)
    return loss

In [11]:
framework_adapter = 'openfl.plugins.frameworks_adapters.pytorch_adapter.FrameworkAdapterPlugin'
model_interface = ModelInterface(model=model_net, optimizer=optimizer, framework_plugin=framework_adapter)

# Save the initial model state
initial_model = deepcopy(model_net)

In [12]:
task_interface = TaskInterface()

# The Interactive API supports registering functions definied in main module or imported.
def function_defined_in_notebook(some_parameter):
    print(f'Also I accept a parameter and it is {some_parameter}')

# Task interface currently supports only standalone functions.
@task_interface.add_kwargs(**{'some_parameter': 42})
@task_interface.register_fl_task(model='net_model', data_loader='train_loader',
                     device='device', optimizer='optimizer') 
#@task_interface.set_aggregation_function(FedCurvWeightedAverage()
def train(net_model, train_loader, optimizer, device, loss_fn=binary_cross_entropy, some_parameter=None):
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.75, patience = 4)
    torch.manual_seed(0)
    #fedcurv.on_train_begin(net_model)
    device='cpu'
    function_defined_in_notebook(some_parameter)
    
    train_loader = tqdm.tqdm(train_loader, desc="train")
    net_model.train()
    net_model.to(device)

    train_loss = 0.0
    epochs = 8
    
    for epoch in range(epochs):
        if epoch == 8 // 2: #Primero se entrenan un poco todo el modelo y luego se realiza 
            net_model.load_state_dict(torch.load('saved_state.pth'))
            for param in net_model.features.parameters():
                param.requires_grad = False
        
        if scheduler != None:
            scheduler.step(train_loss)
        
        net_model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = net_model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()   
            train_loss += loss.item() * data.size(0)   
    return {'train_loss': train_loss,}

@task_interface.register_fl_task(model='net_model', data_loader='val_loader', device='device')     
def validate(net_model, val_loader, device):
    torch.manual_seed(0)
    
    val_loader = tqdm.tqdm(val_loader, desc = "validate")
    net_model.eval()
    metrics = {'Accuracy':[], 'Precision':[], 'Recall':[], 'F1-Score':[]}
    number_correct, number_data = 0, 0
    true_labels, predicted_labels = [], []

    for data, target in val_loader:
        output = net_model(data)
        _, pred = torch.max(output, 1) 
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) 
        number_correct += sum(correct)
        number_data += correct.shape[0]
        true_labels.extend(target.cpu().numpy())
        predicted_labels.extend(pred.cpu().numpy())

    train_loss = train_loss / len(val_loader)
    valid_loss = valid_loss / len(val_loader)
    accuracy = (100 * number_correct / number_data)
    precision, recall, f1_score, support = precision_recall_fscore_support(
        true_labels, predicted_labels, average='weighted')
    
    metrics['Accuracy'].append(accuracy)
    metrics['Precision'].append(precision*100)
    metrics['Recall'].append(recall*100)
    metrics['F1-Score'].append(f1_score*100)  
              
    return metrics

In [13]:
experiment_name = 'ChestXray_EPOCHS8_ROUND10_CNN'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [14]:
# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(
    model_provider=model_interface, 
    task_keeper=task_interface,
    data_loader=fed_dataset,
    rounds_to_train=10,
    opt_treatment='CONTINUE_GLOBAL'
)

[01:34:20] INFO     Building 🡆 Object CloudpickleSerializer from openfl.plugins.interface_serializer.cloudpickle_serializer Module.                  ]8;id=772045;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=917319;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py#173\173]8;;\

           INFO     Building 🡆 Object FrameworkAdapterPlugin from openfl.plugins.frameworks_adapters.pytorch_adapter Module.                         ]8;id=926384;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=207461;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py#173\173]8;;\

C:\Users\clash\AppData\Roaming\Python\Python37\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[01:34:26] INFO     Starting experiment!                                                                                                       ]8;id=907947;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\interface\interactive_api\experiment.py\experiment.py]8;;\:]8;id=912095;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\interface\interactive_api\experiment.py#245\245]8;;\

           INFO     FL-Plan hash is 711156101db858939d65ac398982d7da77f3a5e914dd3c06b333773024bbc67755f03e5683934ad071b831c3ba951b98                 ]8;id=385915;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=835947;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py#236\236]8;;\

...

           INFO     Building 🡆 Object FrameworkAdapterPlugin from openfl.plugins.frameworks_adapters.pytorch_adapter Module.                         ]8;id=803570;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py\plan.py]8;;\:]8;id=847522;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\federated\plan\plan.py#173\173]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=654372;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\utilities\utils.py\utils.py]8;;\:]8;id=444545;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\utilities\utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=282070;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\utilities\utils.py\utils.py]8;;\:]8;id=45411;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\utilities\utils.py#172\172]8;;\

           INFO     SetNewExperiment                                                                                                      ]8;id=126005;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\transport\grpc\director_client.py\director_client.py]8;;\:]8;id=233544;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\transport\grpc\director_client.py#209\209]8;;\

           INFO     Experiment was submitted to the director!                                                                                  ]8;id=25237;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\interface\interactive_api\experiment.py\experiment.py]8;;\:]8;id=215997;file://c:\Users\clash\AppData\Local\Continuum\anaconda3\envs\TFG-FL\lib\site-packages\openfl\interface\interactive_api\experiment.py#259\259]8;;\